In [16]:
###All in One
import imaplib
import email
import time
from email.header import decode_header
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import configparser
import csv
import smtplib
import hashlib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import qrcode
from PIL import Image, ImageDraw, ImageFont
import datetime

# Set up logging configuration
with open('app.log', 'w') as log_file:
    pass

class FileLogger:
    def __init__(self, filename):
        self.filename = filename

    def log(self, message):
        with open(self.filename, 'a') as f:
            f.write(message + '\n')

# Usage
logger = FileLogger('app.log')
logger.log("Starting the script")

#Global
ticketcsv = "test-Teilnehmer.csv"
eventid = "2457"
 # SMTP server configuration
config = configparser.ConfigParser()
config.read('config.ini')
smtp_server = config.get('SMTP', 'server')
smtp_port = config.getint('SMTP', 'port')
smtp_username = config.get('SMTP', 'username')
smtp_password = config.get('SMTP', 'password')
smtp_from = config.get('SMTP', 'from')
imap = config.get('outlook', 'username')
imap = config.get('outlook', 'password')


def getTickets():
    ###Get Tickets from WordPress
    try:
        # Read the credentials from the config file
        username = config.get('credentials', 'username')
        password = config.get('credentials', 'password')

        # Create a new instance of the Chrome driver
        driver = webdriver.Chrome()

        # Go to the specified URL
        driver.get("https://www.mk-waldburg-hannober.de/wp-admin/edit.php?post_type=tribe_events&page=tickets-attendees&event_id=" + eventid)

        # Find the username field, enter the username, and press Enter
        username_field = driver.find_element(By.XPATH, '//*[@id="user_login"]')
        username_field.send_keys(username)

        # Find the password field, enter the password, and press Enter
        password_field = driver.find_element(By.XPATH, '//*[@id="user_pass"]')
        password_field.send_keys(password)
        password_field.send_keys(Keys.RETURN)

        # Delete Old test-ticket.csv
        if os.path.exists(ticketcsv):
            os.remove(ticketcsv)

        # Click on the specified element
        element_to_click = driver.find_element(By.XPATH, '//*[@id="event-tickets__attendees-admin-form"]/div[1]/div[2]/a[1]')
        element_to_click.click()

        # Wait for 5 seconds
        time.sleep(5)

        # Logout
        driver.get("https://www.mk-waldburg-hannober.de/wp-login.php?action=logout&_wpnonce=")
        logout_button = driver.find_element(By.XPATH, '//*[@id="error-page"]/div/p[2]/a')
        logout_button.click()

        # Close the driver
        driver.quit()
        logger.log(f"{datetime.datetime.now()}: Tickets downloaded successfully")

    except Exception as e:
        # Close the driver when Error occured
        #LOGTHIS
        driver.quit()
        logger.log(f"{datetime.datetime.now()}: An error occurred with chrome-driver: {str(e)} at ")

def sendTickets():
    ###Send Tickets QR-Code via email

    # Open the CSV file
    try:

        if os.path.exists('processed_tickets.csv'):
            with open('processed_tickets.csv', 'r') as f:
                processed_tickets = f.read().splitlines()
        else:
            print('No processed_tickets.csv file found, creating an empty one.')
            with open('processed_tickets.csv', 'w') as f:
                pass
            processed_tickets = []  

        with open(ticketcsv, 'r') as f:
            reader = csv.reader(f)
            next(reader)  # Skip the header row

            # For each row in the CSV file
            for row in reader:
                # Extract the ticket ID and recipient email
                ticket_id = row[1]  # The 10-character alphanumeric string
                recipient_email = row[7]

                # Calculate the SHA256 hash of the ticket ID
                ticket_id_hash = hashlib.sha256(ticket_id.encode()).hexdigest()

                

                # If we've already processed this ticket ID, skip
                if ticket_id_hash in processed_tickets:
                    print(f'Ticket ID {ticket_id} already processed')
                    continue

                # Generate a QR code for the ticket ID
                print(f'Generating QR code for ticket ID {ticket_id}')
                qr = qrcode.QRCode(
                    version=1,
                    error_correction=qrcode.constants.ERROR_CORRECT_L,
                    box_size=10,
                    border=4,
                )
                qr.add_data(ticket_id)
                qr.make(fit=True)

                img = qr.make_image(fill='black', back_color='white')
                img = img.convert("RGB")

                # Create a new image with extra space for text
                new_img = Image.new("RGB", (img.width, img.height + 30))

                # Paste the QR code into the new image
                new_img.paste(img, (0, 0))

                # Add text under the QR code
                draw = ImageDraw.Draw(new_img)
                font = ImageFont.load_default()  # Use the default font
                draw.text((15, img.height + 15), ticket_id, font=font, fill=(154,205,50))  # Change text color to red

                filename = f'ticket_{ticket_id}.png'
                new_img.save(filename)

                # Create a multipart message
                msg = MIMEMultipart()
                msg['From'] = smtp_from
                msg['To'] = recipient_email
                msg['Subject'] = 'QR-Code für Ihr Ticket'

                # Attach the QR code image to email
                with open(filename, 'rb') as f:
                    part = MIMEBase('application', 'octet-stream')
                    part.set_payload(f.read())
                    encoders.encode_base64(part)
                    part.add_header('Content-Disposition', f'attachment; filename={filename}')
                    msg.attach(part)

                # Connect to the SMTP server and send the email
                with smtplib.SMTP("smtp.office365.com", 587) as server:
                    server.starttls()
                    server.login(smtp_username, smtp_password)
                    server.send_message(msg)
                
                logger.log(f'{datetime.datetime.now()}: Email sent for ticket ID {ticket_id} to {recipient_email}')

                # Delete the QR code image
                os.remove(filename)

                # Add the processed ticket ID hash to the list of processed tickets
                processed_tickets.append(ticket_id_hash)

            # Save the updated list of processed tickets
            with open('processed_tickets.csv', 'w') as f:
                logger.log(f'{datetime.datetime.now()}: Saving processed tickets to file. Tickets are: {processed_tickets}')
                for ticket in processed_tickets:
                    f.write(f'{ticket}\n')

            os.remove(ticketcsv)

    except Exception as e:
        logger.log(f"{datetime.datetime.now()}: An error occurred while sending tickets: {str(e)}")

while True:
    # create an IM4 class with SSL 
    mail = imaplib.IMAP4_SSL("outlook.office365.com")
    # authenticate
    mail.login(imap_username, imap_password)

    # select the mailbox you want to delete in
    # if you want SPAM, use "INBOX.SPAM"
    mailbox = "INBOX"
    mail.select(mailbox)

    # search for specific mail
    resp, items = mail.search(None, "(UNSEEN)")
    items = items[0].split()

    # if there are any new mails
    if items:
        logger.log(f"{datetime.datetime.now()}: New tickets available")
        #LOGTHIS
        try:
            getTickets()
            time.sleep(3)
            sendTickets()
            for emailid in items:
                mail.store(emailid, '+FLAGS', '\Seen')  # Mark as read

        except:
            logger.log(f"{datetime.datetime.now()}: Error occurred in while loop getting mail")
        
    else:
        print("No new tickets")
    
    # wait for 60 seconds
    time.sleep(10)

No new tickets
Generating QR code for ticket ID 7766bb3b89
Generating QR code for ticket ID 993980b1bc
Generating QR code for ticket ID b5e159c59a
No new tickets
No new tickets
No new tickets
